# 🎮 Steam ACF Generator for Google Colab

**Generate Steam App Configuration Files (ACF) easily!**

This tool uses SKSAppManifestGenerator to create ACF files for your Steam App IDs.

## 📋 How to Use:
1. **Install Wine** (if needed) - Run the cell below
2. **Install Dependencies** - Run the next cell  
3. **Configure & Generate** - Use the interactive form below

## ⚠️ Important Notes:
- **Windows Tool**: SKSAppManifestGenerator is a Windows executable (.exe)
- **Colab Limitation**: Google Colab runs on Linux, so Wine may be needed
- **Best Option**: For best results, use the PowerShell script on Windows

## 🙏 Credits
- Original tool: **SKSAppManifestGenerator** by [Sak32009](https://github.com/Sak32009/SKSAppManifestGenerator)
- This wrapper provides a Python interface for Colab users


In [ ]:
# Install Wine if needed (for Linux/Colab to run Windows executables)
# Uncomment the line below if Wine is not available
# !apt-get update && apt-get install -y wine


In [ ]:
!pip install requests ipywidgets -q


In [ ]:
#@title Steam ACF Generator UI (Form)
# Hidden code - Setup and functions
import os
import sys
import requests
import zipfile
import subprocess
import shutil
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Configuration
PRIMARY_URL = 'https://github.com/Sak32009/SKSAppManifestGenerator/releases/download/v2.0.3/SKSAppManifestGenerator_x64_v2.0.3.zip'
FALLBACK_URL = 'https://github.com/ahmed98Osama/Steam-acf-generator/raw/master/SKSAppManifestGenerator_x64.exe'
TOOL_DIR = './tools/SKSAppManifestGenerator'
TOOL_NAME = 'SKSAppManifestGenerator_x64.exe'
TOOL_PATH = os.path.join(TOOL_DIR, TOOL_NAME)

# Colors for output
class Colors:
    CYAN = '\033[96m'
    YELLOW = '\033[93m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    GRAY = '\033[90m'
    RESET = '\033[0m'

def print_info(msg):
    print(f"{Colors.CYAN}[INFO]{Colors.RESET} {msg}")

def print_warn(msg):
    print(f"{Colors.YELLOW}[WARN]{Colors.RESET} {msg}")

def print_err(msg):
    print(f"{Colors.RED}[ERROR]{Colors.RESET} {msg}")

def print_success(msg):
    print(f"{Colors.GREEN}[SUCCESS]{Colors.RESET} {msg}")

def download_file(url, output_path):
    """Download a file from URL"""
    try:
        print_info(f"Downloading from: {url}")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        percent = (downloaded / total_size) * 100
                        print(f"\r  Progress: {percent:.1f}%", end='', flush=True)
        print()
        return True
    except Exception as e:
        print_warn(f"Download failed: {str(e)}")
        return False

def extract_zip(zip_path, extract_to):
    """Extract ZIP file"""
    try:
        print_info(f"Extracting {zip_path} to {extract_to}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        return True
    except Exception as e:
        print_warn(f"Extraction failed: {str(e)}")
        return False

def find_executable(directory):
    """Find the executable in extracted directory"""
    for root, dirs, files in os.walk(directory):
        if TOOL_NAME in files:
            return os.path.join(root, TOOL_NAME)
    return None

def setup_tool():
    """Download and setup SKSAppManifestGenerator"""
    if os.path.exists(TOOL_PATH):
        print_info(f"Tool found at: {TOOL_PATH}")
        return TOOL_PATH
    
    print_warn(f"Tool not found at: {TOOL_PATH}")
    os.makedirs(TOOL_DIR, exist_ok=True)
    
    temp_zip = '/tmp/sks_generator.zip'
    temp_extract = '/tmp/sks_generator_extract'
    
    print_info("Attempting download from primary source...")
    if download_file(PRIMARY_URL, temp_zip):
        os.makedirs(temp_extract, exist_ok=True)
        if extract_zip(temp_zip, temp_extract):
            exe_path = find_executable(temp_extract)
            if exe_path:
                shutil.copy2(exe_path, TOOL_PATH)
                os.chmod(TOOL_PATH, 0o755)
                os.remove(temp_zip)
                shutil.rmtree(temp_extract, ignore_errors=True)
                print_success(f"Tool installed at: {TOOL_PATH}")
                return TOOL_PATH
    
    print_info("Primary download failed. Attempting fallback source...")
    if download_file(FALLBACK_URL, TOOL_PATH):
        os.chmod(TOOL_PATH, 0o755)
        print_success(f"Tool installed from fallback at: {TOOL_PATH}")
        return TOOL_PATH
    
    print_err("Failed to download tool from both sources")
    return None

def validate_app_ids(app_ids_input):
    """Validate and normalize App IDs"""
    if not app_ids_input or not app_ids_input.strip():
        return []
    
    tokens = app_ids_input.replace(',', ' ').split()
    app_ids = []
    
    for token in tokens:
        token = token.strip()
        if token.isdigit():
            app_ids.append(token)
        elif token:
            print_warn(f"Skipping invalid App ID: {token}")
    
    return app_ids

def generate_acf_files(tool_path, app_ids, debug=False, working_dir=None):
    """Generate ACF files for given App IDs"""
    if working_dir is None:
        working_dir = os.getcwd()
    
    os.makedirs(working_dir, exist_ok=True)
    
    cmd = [tool_path]
    if debug:
        cmd.append('-d')
    cmd.extend(app_ids)
    
    print_info(f"Generating ACF files for App IDs: {', '.join(app_ids)}")
    print_info(f"Working directory: {working_dir}")
    
    original_dir = os.getcwd()
    try:
        os.chdir(working_dir)
        
        print_info("Attempting to execute generator...")
        wine_available = shutil.which('wine') is not None
        
        if wine_available:
            print_info("Wine detected. Using Wine to run Windows executable.")
            cmd = ['wine'] + cmd
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
            if result.returncode == 0:
                print_success("ACF files generated successfully!")
                print(result.stdout)
            else:
                print_warn(f"Generator returned code {result.returncode}")
                print(result.stderr)
        except FileNotFoundError:
            print_err("Cannot execute Windows executable on Linux.")
            print_info("Please use the PowerShell script on Windows, or install Wine.")
        except subprocess.TimeoutExpired:
            print_err("Generator timed out")
    
    finally:
        os.chdir(original_dir)
    
    found_files = []
    for app_id in app_ids:
        candidates = [
            os.path.join(working_dir, f"appmanifest_{app_id}.acf"),
            os.path.join(working_dir, f"{app_id}.acf")
        ]
        for candidate in candidates:
            if os.path.exists(candidate):
                found_files.append(candidate)
                break
    
    if found_files:
        print_success(f"Found {len(found_files)} generated file(s):")
        for f in found_files:
            print(f"  - {f}")
    else:
        print_warn("No ACF files detected. Check generator output above.")

# Global variables for widgets
tool_path = None
status_output = widgets.Output()

def on_generate_clicked(b):
    """Handle generate button click"""
    with status_output:
        clear_output()
        
        # Get values from widgets
        app_ids_input = app_id_input.value
        debug = debug_checkbox.value
        working_dir = working_dir_input.value if working_dir_input.value else os.getcwd()
        
        # Validate App IDs
        app_ids = validate_app_ids(app_ids_input)
        if not app_ids:
            print_err("No valid App IDs provided.")
            return
        
        print(f"{Colors.GREEN}Valid App IDs:{Colors.RESET} {', '.join(app_ids)}")
        
        # Setup tool if needed
        global tool_path
        if not tool_path:
            print_info("Setting up SKSAppManifestGenerator...")
            tool_path = setup_tool()
            if not tool_path:
                print_err("Could not setup SKSAppManifestGenerator. Exiting.")
                return
        
        # Generate files
        print("\n" + "=" * 50)
        generate_acf_files(tool_path, app_ids, debug, working_dir)
        
        print("\n" + "=" * 50)
        print_success("Process completed!")
        print("=" * 50 + "\n")

# Create widgets
app_id_input = widgets.Text(
    value='',
    placeholder='Enter App IDs separated by commas or spaces (e.g., 570, 730, 440)',
    description='App IDs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

debug_checkbox = widgets.Checkbox(
    value=False,
    description='Enable debug output',
    style={'description_width': 'initial'}
)

working_dir_input = widgets.Text(
    value='',
    placeholder=f'Leave empty to use current directory: {os.getcwd()}',
    description='Working Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

generate_button = widgets.Button(
    description='🚀 Generate ACF Files',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

generate_button.on_click(on_generate_clicked)

# Display the interface
display(HTML("""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; margin: 10px 0;">
    <h2 style="color: white; text-align: center; margin: 0;">🎮 Steam ACF Generator</h2>
    <p style="color: white; text-align: center; margin: 10px 0;">Generate Steam App Configuration Files easily!</p>
</div>
"""))

display(HTML("<h3>📝 Configuration</h3>"))
display(app_id_input)
display(debug_checkbox)
display(working_dir_input)

display(HTML("<br>"))
display(generate_button)

display(HTML("<h3>📊 Status & Output</h3>"))
display(status_output)
